<a href="https://colab.research.google.com/github/fangyiyu/Fangyi_Yu_Queens/blob/master/Modified_MNIST_TF1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.__version__

'2.3.0'

In [1]:
!pip install tensorflow==1.13.1

     |████████████████████████████████| 92.5MB 31kB/s 
     |████████████████████████████████| 3.2MB 52.1MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 368kB 56.7MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.contrib import rnn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [3]:
!wget -O comp551w18-modified-mnist.zip https://www.dropbox.com/sh/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a?dl=0

--2020-09-28 21:48:41--  https://www.dropbox.com/sh/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a [following]
--2020-09-28 21:48:41--  https://www.dropbox.com/sh/raw/dhu4cu8l8e32cvl/AACSbGE9X6P7an61STwwr8R0a
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3faec4e1bd1e2db0bc38bb50e5.dl.dropboxusercontent.com/zip_download_get/Ajqqu0R-d-JiVdTlc4ExHX75a1WEZzeybFROk_zdTaM3UBf8IsqIdM1RR7V2GESIAGN6CLVgeEBkO1PgU_oQAc8Un_CYOG-09qyit5MONAlMLA [following]
--2020-09-28 21:48:42--  https://uc3faec4e1bd1e2db0bc38bb50e5.dl.dropboxusercontent.com/zip_download_get/Ajqqu0R-d-JiVdTlc4ExHX75a1WEZzeybFROk_zdTaM3UBf8IsqIdM1RR7V2GESIAGN6CLVgeEBkO1PgU

In [5]:
!unzip comp551w18-modified-mnist.zip

Archive:  comp551w18-modified-mnist.zip
  inflating: test_x.csv              
  inflating: train_y.csv             
  inflating: train_x.csv             


# 1. Data preprocessing

In [6]:
# Load data
x = np.loadtxt("train_x.csv", delimiter=",")# load from text
y = np.loadtxt("train_y.csv", delimiter=",")
x_train = x.reshape(-1, 64, 64) # reshape
y_train = y.reshape(-1, 1)

In [7]:
# Scale and reshape x
x_train = x_train/255
x_train = x_train.reshape(len(x_train),64*64)

In [8]:
# Transform y
onehot = OneHotEncoder(sparse = False)
y_train = onehot.fit_transform(y_train)

In [10]:
# Split dataset into training and testing subsets
X_train, X_test, Y_train, Y_test = train_test_split(x_train,y_train,test_size=0.2, random_state = 17)

## 2. Logistic Regression

In [15]:
import tensorflow as tf
# Parameters
learning_rate = 0.01
training_epochs = 50
batch_size = 256
display_step = 50

# TF graph input
x = tf.placeholder(tf.float32, [None, 4096]) # mnist data image of shape 64*64=4096
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes


In [16]:
# Set model weights                                                   
W = tf.Variable(tf.zeros([4096, 10]))
b = tf.Variable(tf.zeros([10]))

# Construct model
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

# Minimize error using cross entropy
cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [17]:
# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(X_train)/batch_size)
        n = 0
        # Loop over all batches
        for i in range(total_batch):
            if n+batch_size >= X_train.shape[0]:
              n = 0
            batch_xs = X_train[n:n+batch_size]
            batch_ys = Y_train[n:n+batch_size]
            n += batch_size
            
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            
            # Compute average loss
            avg_cost += c / total_batch
            
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: X_test, y: Y_test}))

Epoch: 0050 cost= 2.191021394
Optimization Finished!
Accuracy: 0.1275


## 3. CNN

In [63]:
# Training Parameters
learning_rate = 0.01
num_steps = 100
batch_size = 256
display_step = 50

# Network Parameters
num_input = 4096
num_classes = 10 
dropout = 0.7 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [64]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 4096 features (64*64 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 64, 64, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    
    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling)
    conv3 = maxpool2d(conv3, k=2)
 
    # Fully connected layer
    # Reshape conv3 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [65]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 3x3 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([3, 3, 32, 64])),
    # 3x3 conv, 64 inputs, 128 outputs
    'wc3': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    # fully connected, 8*8*128 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([8*8*128, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([128])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [66]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    n = 0
    for step in range(1, num_steps+1):
        if n+batch_size >= X_train.shape[0]:
          n = 0
        batch_x = X_train[n:n+batch_size]
        batch_y = Y_train[n:n+batch_size]
        n = n+batch_size
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 1000 test data points
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test[:1000],
                                      Y: Y_test[:1000],
                                      keep_prob: 1.0}))

Step 1, Minibatch Loss= 2526326.5000, Training Accuracy= 0.105
Step 50, Minibatch Loss= 8017.8306, Training Accuracy= 0.105
Step 100, Minibatch Loss= 645.2643, Training Accuracy= 0.098
Optimization Finished!
Testing Accuracy: 0.124


## 4. RNN

In [67]:
tf.reset_default_graph()

# Training Parameters
learning_rate = 0.01
training_steps = 100
batch_size = 256
display_step = 50

# Network Parameters
num_input = 64 # data input: 64x64
timesteps = 64 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [68]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [69]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [70]:
logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [71]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    n = 0
    for step in range(1, training_steps+1):
        if n+batch_size >= X_train.shape[0]:
          n= 0 
        batch_x = X_train[n:n+batch_size]
        batch_y = Y_train[n:n+batch_size]
        n = n+batch_size
        # Reshape data to get 64 seq of 64 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 1000 test data points
    test_data = X_test[:1000].reshape((-1, timesteps, num_input))
    test_label = Y_test[:1000]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 11.2199, Training Accuracy= 0.094
Step 50, Minibatch Loss= 2.3380, Training Accuracy= 0.094
Step 100, Minibatch Loss= 2.3166, Training Accuracy= 0.098
Optimization Finished!
Testing Accuracy: 0.121


## 5. Gated CNN

In [54]:
# Training Parameters
learning_rate = 0.01
num_steps = 100
batch_size = 256
display_step = 50

# Network Parameters
num_input = 4096
num_classes = 10 #total classes (0-9 digits)
dropout = 0.7 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [55]:
def conv2d(x, W, b, V, c, num, strides=1):
    # Conv2D wrapper, with bias and gates
    A = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    B = tf.nn.conv2d(x, V, strides=[1, strides, strides, 1], padding='SAME')
    A = tf.nn.bias_add(A, b)
    B = tf.nn.bias_add(B, c)
    return tf.multiply(A, tf.nn.sigmoid(B))


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    
    x = tf.reshape(x, shape=[-1, 64, 64, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'], weights['wv1'], biases['bv1'], weights['c1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'], weights['wv2'], biases['bv2'], weights['c2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'], weights['wv3'], biases['bv3'], weights['c3'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv3, k=2)

    
    
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv3, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [59]:
# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wv1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'c1': 5,
    
    'wc2': tf.Variable(tf.random_normal([3, 3, 32, 64])),
    'wv2': tf.Variable(tf.random_normal([3, 3, 32, 64])),
    'c2': 3,
    
    'wc3': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'wv3': tf.Variable(tf.random_normal([3, 3, 64, 128])),
    'c3': 3,
 
    'wd1': tf.Variable(tf.random_normal([16*16*128, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bv1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bv2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([128])),
    'bv3': tf.Variable(tf.random_normal([128])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [60]:
# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [61]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    n=0
    for step in range(1, num_steps+1):
        if n+batch_size >= X_train.shape[0]:
          n=0
        batch_x = X_train[n:n+batch_size]
        batch_y = Y_train[n:n+batch_size]
        n = n+batch_size
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 1000 test data points
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test[:1000],
                                      Y: Y_test[:1000],
                                      keep_prob: 1.0}))

Step 1, Minibatch Loss= 6339567.0000, Training Accuracy= 0.109
Step 50, Minibatch Loss= 8965.8340, Training Accuracy= 0.133
Step 100, Minibatch Loss= 180.1911, Training Accuracy= 0.125
Optimization Finished!
Testing Accuracy: 0.111
